# Import Libraries

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime as dt
import itertools
from IPython.display import display


pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.precision', 3)
pd.set_option('display.width', 3000)
pd.set_option('display.max_columns', 75)
pd.set_option('display.max_rows', 38)

In [3]:
from sklearn.metrics import classification_report,  confusion_matrix, accuracy_score, f1_score,  precision_recall_fscore_support
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, accuracy_score, roc_auc_score, f1_score


In [4]:
folder='BPL Football Data/'

In [5]:
# Read data from the CSV into a dataframe

raw_data={}
for year in range(2000,2022):
  season = f'{year}-{str(year+1)[2:]}.csv'
  raw_data[year] = pd.read_csv(folder+season)

In [6]:
# Columns to start:
cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC',
       'AC', 'HY', 'AY', 'HR', 'AR']


In [7]:
stats = {}

for year in raw_data.keys():
  stats[year] = raw_data[year][cols]


# Feature Engineering

#Stats and Feature Engineering
**Will get the average later**

HTGS: Home Total Goals Scored   
ATGS: Away Total Goals Scored  
HTGC: Home Total Goals Conceded    
ATGC: Away Total Goals Conced  
HTS: Home Total Shots  
ATS: Away Total Shots  
HTSC: Home total shots conceded   
ATSC: Away total shots conceded   
HTTS: Home Total Target Shots  
ATTS: Away Total Target Shots  
HTTSC: Home total target shots conceded  
ATTSC: Away total target shots conceded   
HTF: Home Total Fouls  
ATF: Away Total Fouls  
HTC: Home Total Corners  
ATC: Away Total Corners  
HTY: Home Total Yellows  
ATY: Away Total Yellows  
HTR: Home Total Reds  
ATR: Home Total Reds  
HTP: Home Team Points   
ATP: Away Team Points   
HTFormPts: Home points in the last 5 games   
ATFormPts:  Aeay points in the last 5 games  
MW: Matchweek   

In [8]:
def in_game_stats(year_stats):

  dfs = {}

  for team in year_stats['HomeTeam'].unique(): # loops through all the teams in a season in a data frame and puts it in a dictionary where the key is the team name and the value is the dataframe

    home_df  = year_stats[year_stats['HomeTeam']==team]   
    away_df  = year_stats[year_stats['AwayTeam']==team]


    comb_df = pd.concat([home_df, away_df]).sort_index()

    def off_cums(row):
      '''Gets the offensive stats of team'''
      if row[0] == team:
        return row[1]
      else: 
        return row[2]

    def def_cums(row):
      '''Gets defensive stats of team'''
      if row[0] == team:
        return row[2]
      else: 
        return row[1]

    def get_points(row):
      '''Gets the offensive stats of team'''
      if row['HomeTeam'] == team:
        if row['FTHG'] > row['FTAG']:
          return 3
        if row['FTHG'] == row['FTAG']:
          return 1
        else:
          return 0
      else:
        if row['FTHG'] < row['FTAG']:
          return 3
        if row['FTHG'] == row['FTAG']:
          return 1
        else:
          return 0



    old_cols = ['FTHG', 'FTAG', 'HS', 'AS', 'HST', 'AST']
    new_cols = ['TGS', 'TGC', 'TS','TSC', 'TTS','TTSC']


    for i in (range(0, len(old_cols),2)): 
      home_stat = old_cols[i]
      away_stat = old_cols[i+1]
      comb_df[new_cols[i]]= comb_df[['HomeTeam',home_stat,away_stat]].apply(off_cums, axis=1).cumsum().shift(periods=1, fill_value=0)       # gets the running sum of all the offensive stats
      comb_df[new_cols[i+1]]= comb_df[['HomeTeam',home_stat,away_stat]].apply(def_cums, axis=1).cumsum().shift(periods=1, fill_value=0)     # gets the runing sum of all the defensive stats 


    old_cols = ['HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR',]
    new_cols = ['TF', 'TC', 'TY', 'TR']

    for i in (range(0, len(old_cols),2)):
      home_stat = old_cols[i]
      away_stat = old_cols[i+1]
      comb_df[new_cols[int(i/2)]]= comb_df[['HomeTeam',home_stat,away_stat]].apply(off_cums, axis=1).cumsum().shift(periods=1, fill_value=0) # gets the running summ of all the miscellenous stats

    comb_df['TP']= comb_df[['HomeTeam','FTHG','FTAG']].apply(get_points, axis=1).cumsum().shift(periods=1, fill_value=0) # gets cumaltive points 
    comb_df['TFormPts'] = comb_df['TP'] - comb_df['TP'].shift(5,fill_value=0) # gets the points from the last 5 games
    comb_df['Team'] = team

    dfs[team] = comb_df 

  df = pd.concat([dfs[team] for team in year_stats['HomeTeam'].unique()]).sort_index() # concats all the dataframes in the dfs dictionary 
  df1 = df[df['HomeTeam'] == df['Team']]
  df2 = df[df['AwayTeam'] == df['Team']]
  final_year = df1.drop(columns='Team').merge(df2.iloc[:,19:-1], how='inner', right_index=True, left_index=True, suffixes=['H','A']) 

  new_cols = [col[-1]+col[:-1] for col in final_year.columns[19:]]
  old_cols = final_year.columns[19:]
  col_dic = dict(zip(old_cols,new_cols)) # renames columns for clarity

  final_year =final_year.rename(columns=col_dic)

  return final_year




In [9]:
stats1 = {}
for year in stats.keys():
  stats1[year] = in_game_stats(stats.copy()[year])

## Get MatchWeek:

In [10]:
def get_mw(playing_stat):
    j = 1
    MatchWeek = []
    for i in range(380):
        MatchWeek.append(j)
        if ((i + 1)% 10) == 0:
            j = j + 1
    playing_stat['MW'] = MatchWeek
    return playing_stat

stats2 = {}
for year in stats.keys():
  stats2[year] = get_mw(stats1[year].copy())


In [11]:
stats2[2017]

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,HTGS,HTGC,HTS,HTSC,HTTS,HTTSC,HTF,HTC,HTY,HTR,HTP,HTFormPts,ATGS,ATGC,ATS,ATSC,ATTS,ATTSC,ATF,ATC,ATY,ATR,ATP,ATFormPts,MW
0,11/08/17,Arsenal,Leicester,4,3,H,M Dean,27,6,10,3,9,12,9,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,12/08/17,Brighton,Man City,0,2,A,M Oliver,6,14,2,4,6,9,3,10,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,12/08/17,Chelsea,Burnley,2,3,A,C Pawson,19,10,6,5,16,11,8,5,3,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,12/08/17,Crystal Palace,Huddersfield,0,3,A,J Moss,14,8,4,6,7,19,12,9,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,12/08/17,Everton,Stoke,1,0,H,N Swarbrick,9,9,4,1,13,10,6,7,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,13/05/18,Newcastle,Chelsea,3,0,H,M Atkinson,16,6,6,2,11,10,4,2,0,1,0,0,36,47,433,473,144,155,398,163,52,2,41,3,62,35,600,361,218,114,353,228,40,4,70,13,38
376,13/05/18,Southampton,Man City,0,1,A,A Marriner,8,13,3,2,8,10,1,12,3,1,0,0,37,55,443,480,142,168,408,226,60,2,36,8,105,27,651,230,259,84,334,272,57,2,97,13,38
377,13/05/18,Swansea,Stoke,1,2,A,A Taylor,26,8,11,5,12,9,6,0,1,2,0,0,27,54,310,530,93,186,365,144,50,1,33,1,33,67,374,552,126,209,427,136,60,1,30,3,38
378,13/05/18,Tottenham,Leicester,5,4,H,C Pawson,14,16,6,9,9,13,4,4,1,2,0,0,69,32,608,339,211,117,375,242,48,2,74,7,52,55,406,476,139,162,352,199,50,5,47,4,38


**GET TEAM FORM:**

In [ ]:
def get_matchres(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []

    # the value corresponding to keys is a list containing the match result
    for i in range(len(playing_stat)):
        if playing_stat.iloc[i].FTR == 'H':
            teams[playing_stat.iloc[i].HomeTeam].append('W')
            teams[playing_stat.iloc[i].AwayTeam].append('L')
        elif playing_stat.iloc[i].FTR == 'A':
            teams[playing_stat.iloc[i].AwayTeam].append('W')
            teams[playing_stat.iloc[i].HomeTeam].append('L')
        else:
            teams[playing_stat.iloc[i].AwayTeam].append('D')
            teams[playing_stat.iloc[i].HomeTeam].append('D')
            
    return pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T

def get_form(playing_stat,num):
    form = get_matchres(playing_stat)
    form_final = form.copy()
    for i in range(num,39):
        form_final[i] = ''
        j = 0
        while j < num:
            form_final[i] += form[i-j]
            j += 1           
    return form_final

def add_form(playing_stat,num):
    form = get_form(playing_stat,num)
    h = ['M' for i in range(num * 10)]  # since form is not available for n MW (n*10)
    a = ['M' for i in range(num * 10)]
    
    j = num
    for i in range((num*10),380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        
        past = form.loc[ht][j]               # get past n results
        h.append(past[num-1])                    # 0 index is most recent
        
        past = form.loc[at][j]               # get past n results.
        a.append(past[num-1])                   # 0 index is most recent
        
        if ((i + 1)% 10) == 0:
            j = j + 1

    playing_stat['HM' + str(num)] = h                 
    playing_stat['AM' + str(num)] = a

    
    return playing_stat


def add_form_df(playing_statistics):
    playing_statistics = add_form(playing_statistics,1)
    playing_statistics = add_form(playing_statistics,2)
    playing_statistics = add_form(playing_statistics,3)
    playing_statistics = add_form(playing_statistics,4)
    playing_statistics = add_form(playing_statistics,5)
    return playing_statistics    
    
# Make changes to df
stats3 = {}
for year in stats.keys():
  stats3[year] = add_form_df(stats2[year].copy())

In [13]:
stats3[2017][:5]

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,HTGS,HTGC,HTS,HTSC,HTTS,HTTSC,HTF,HTC,HTY,HTR,HTP,HTFormPts,ATGS,ATGC,ATS,ATSC,ATTS,ATTSC,ATF,ATC,ATY,ATR,ATP,ATFormPts,MW,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5
0,11/08/17,Arsenal,Leicester,4,3,H,M Dean,27,6,10,3,9,12,9,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,M,M,M,M,M,M,M,M,M,M
1,12/08/17,Brighton,Man City,0,2,A,M Oliver,6,14,2,4,6,9,3,10,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,M,M,M,M,M,M,M,M,M,M
2,12/08/17,Chelsea,Burnley,2,3,A,C Pawson,19,10,6,5,16,11,8,5,3,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,M,M,M,M,M,M,M,M,M,M
3,12/08/17,Crystal Palace,Huddersfield,0,3,A,J Moss,14,8,4,6,7,19,12,9,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,M,M,M,M,M,M,M,M,M,M
4,12/08/17,Everton,Stoke,1,0,H,N Swarbrick,9,9,4,1,13,10,6,7,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,M,M,M,M,M,M,M,M,M,M


## Convert to DataFrame


In [14]:
weeks_skip = 4 # removes first weeks of each season
stats_df = pd.concat([stats3[year][weeks_skip*10:] for year in stats3.keys()], ignore_index=True)
whole_stats = pd.concat([stats3[year] for year in stats3.keys()], ignore_index=True)

## Get Odds Data

In [ ]:
cols_odds = ['B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'GBH', 'GBD','GBA',
             'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD',  'PSA', 'WHH',
             'WHD', 'WHA', 'SJH', 'SJD', 'SJA', 'VCH', 'VCD', 'VCA', 'BSH', 'BSD', 'BSA']
odds_data={}
for year in raw_data.keys():

  odds_data[year] = raw_data[year]        
  cols = ['Date','HomeTeam','AwayTeam'] + list(set(odds_data[year].columns).intersection(cols_odds))
  odds_data[year] = odds_data[year][cols]
  c = odds_data[year].columns
  cols =  ['Date','HomeTeam','AwayTeam']+ [i for i in c if i[-1] == 'H']
  odds_data[year] = odds_data[year][cols]
  odds_data[year]['AVG Home Odds'] = odds_data[year].mean(axis=1)
  
odds_df = pd.concat([odds_data[year][weeks_skip*10:] for year in odds_data.keys()], ignore_index=True)
odds_df = odds_df[['AVG Home Odds','IWH', 'GBH', 'LBH', 'WHH', 'B365H', 'BWH', 'SJH', 'VCH', 'BSH', 'PSH']]

whole_odds_df = pd.concat([odds_data[year] for year in odds_data.keys()], ignore_index=True)
whole_odds_df = whole_odds_df[['AVG Home Odds','IWH', 'GBH', 'LBH', 'WHH', 'B365H', 'BWH', 'SJH', 'VCH', 'BSH', 'PSH']]


In [17]:
stats_df = pd.merge(stats_df, odds_df, right_index=True, left_index=True)
whole_stats= pd.merge(whole_stats, odds_df, right_index=True, left_index=True)

In [24]:
# stats_df.to_csv('Clean Data\Clean_data.csv')
# whole_stats.to_csv('Clean Data\Whole_Clean_data.csv')